# OOM errors with scan_parquet > limit > collect

See https://github.com/pola-rs/polars/issues/9001

In [1]:
import polars as pl
import pyarrow.dataset as ds

In [2]:
pl.show_versions()

--------Version info---------
Polars:               0.19.18
Index type:           UInt32
Platform:             Linux-6.1.0-13-amd64-x86_64-with-glibc2.36
Python:               3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]

----Optional dependencies----
adbc_driver_manager:  <not installed>
cloudpickle:          3.0.0
connectorx:           <not installed>
deltalake:            <not installed>
fsspec:               2023.10.0
gevent:               <not installed>
matplotlib:           3.8.2
numpy:                1.26.2
openpyxl:             <not installed>
pandas:               2.1.3
pyarrow:              14.0.1
pydantic:             <not installed>
pyiceberg:            <not installed>
pyxlsb:               <not installed>
sqlalchemy:           <not installed>
xlsx2csv:             <not installed>
xlsxwriter:           <not installed>


In [3]:
# 635m row dataset
scan_path = "../../test_result.parquet/*.parquet"

## Eager

In [4]:
df = pl.read_parquet(scan_path, n_rows=1000)
df.limit(5)

test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [5]:
df = pl.read_parquet("../../test_result.parquet/part.0.parquet", use_pyarrow=True)
df.limit(5)

test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [6]:
df.schema

OrderedDict([('test_id', Int64),
             ('vehicle_id', Int64),
             ('test_date', Datetime(time_unit='us', time_zone=None)),
             ('test_class_id', Int64),
             ('test_type', Utf8),
             ('test_result', Utf8),
             ('test_mileage', Int64),
             ('postcode_area', Utf8),
             ('make', Utf8),
             ('model', Utf8),
             ('colour', Utf8),
             ('fuel_type', Utf8),
             ('cylinder_capacity', Int64),
             ('first_use_date', Datetime(time_unit='us', time_zone=None))])

## Lazy

In [7]:
lf = pl.scan_parquet(scan_path, n_rows=1000)
lf.schema

OrderedDict([('test_id', Int64),
             ('vehicle_id', Int64),
             ('test_date', Datetime(time_unit='us', time_zone=None)),
             ('test_class_id', Int64),
             ('test_type', Utf8),
             ('test_result', Utf8),
             ('test_mileage', Int64),
             ('postcode_area', Utf8),
             ('make', Utf8),
             ('model', Utf8),
             ('colour', Utf8),
             ('fuel_type', Utf8),
             ('cylinder_capacity', Int64),
             ('first_use_date', Datetime(time_unit='us', time_zone=None))])

In [8]:
lf = pl.scan_pyarrow_dataset(ds.dataset("../../test_result.parquet", format="parquet"))
lf.schema

OrderedDict([('test_id', Int64),
             ('vehicle_id', Int64),
             ('test_date', Datetime(time_unit='us', time_zone=None)),
             ('test_class_id', Int64),
             ('test_type', Utf8),
             ('test_result', Utf8),
             ('test_mileage', Int64),
             ('postcode_area', Utf8),
             ('make', Utf8),
             ('model', Utf8),
             ('colour', Utf8),
             ('fuel_type', Utf8),
             ('cylinder_capacity', Int64),
             ('first_use_date', Datetime(time_unit='us', time_zone=None))])

### Collect

In [9]:
%%time
lf.limit(5).collect()

CPU times: user 373 ms, sys: 104 ms, total: 478 ms
Wall time: 54.3 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [10]:
%%time
# Streaming is faster
lf.limit(5).collect(streaming=True)

CPU times: user 799 ms, sys: 145 ms, total: 945 ms
Wall time: 91.1 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


### Fetch

In [11]:
%%time
lf.limit(5).fetch(n_rows=5)

CPU times: user 766 ms, sys: 89.1 ms, total: 855 ms
Wall time: 81.8 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [12]:
%%time
# More speed!
lf.limit(5).fetch(streaming=True)

CPU times: user 814 ms, sys: 47.8 ms, total: 862 ms
Wall time: 83.3 ms


<timed eval>:2: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


## Issue report examples

In [13]:
%%time
# OOMs No More :-)
pl.scan_parquet(scan_path).limit(5).collect()

CPU times: user 160 ms, sys: 10.7 ms, total: 171 ms
Wall time: 19.5 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [14]:
%%time
pl.scan_parquet(scan_path, n_rows=100).limit(5).collect()

CPU times: user 26.5 ms, sys: 3.24 ms, total: 29.7 ms
Wall time: 12.3 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [15]:
%%time
# Now fast, no OOM
pl.scan_parquet(scan_path, n_rows=100).limit(5).collect(streaming=True)

CPU times: user 15.3 ms, sys: 6.56 ms, total: 21.9 ms
Wall time: 13.3 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [16]:
%%time
# OOMs No More :-)
pl.scan_parquet(scan_path, n_rows=1000000000).limit(5).collect(streaming=True)

CPU times: user 11.1 ms, sys: 7.62 ms, total: 18.7 ms
Wall time: 6.57 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [17]:
%%time
# OOMs no more :-)
pl.scan_parquet(scan_path).limit(5).collect(streaming=True)

CPU times: user 20.6 ms, sys: 0 ns, total: 20.6 ms
Wall time: 5.68 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00


In [18]:
%%time
# OOMs No More :-)
pl.scan_parquet(scan_path, n_rows=1000000000).limit(5).collect()

CPU times: user 7.7 ms, sys: 4.49 ms, total: 12.2 ms
Wall time: 10.8 ms


test_id,vehicle_id,test_date,test_class_id,test_type,test_result,test_mileage,postcode_area,make,model,colour,fuel_type,cylinder_capacity,first_use_date
i64,i64,datetime[μs],i64,str,str,i64,str,str,str,str,str,i64,datetime[μs]
804664368,256274986,2005-01-01 00:00:00,0,"""NT""","""P""",23459,"""TF""","""FORD""","""UNCLASSIFIED""","""SILVER""","""PE""",null,null
392603376,633988704,2005-01-01 00:00:00,0,"""NT""","""P""",40961,"""E""","""LOTUS""","""UNCLASSIFIED""","""RED""","""PE""",null,null
1894843206,1320781748,2005-01-01 00:00:00,0,"""NT""","""P""",16416,"""S""","""VAUXHALL""","""UNCLASSIFIED""","""BLUE""","""PE""",null,null
830908928,1263031090,2005-01-01 00:00:00,4,"""NT""","""P""",93318,"""W""","""LAND ROVER""","""109 V8 S.W.""","""BLUE""","""PE""",3528,1981-04-06 00:00:00
727535460,1123257842,2005-01-01 00:00:00,4,"""NT""","""P""",121930,"""RG""","""CITROEN""","""AX""","""WHITE""","""DI""",1360,1993-08-31 00:00:00
